# Creating programs to determine the elements in a black box

## Importing relevant libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math

## Module for ensuring arrays have equal number of elements

In [ ]:
def array_check(V_1,V_2,w,theta):
    #V_1 - channel 1 voltage one
    #V_2 - channel 2 voltage two
    #w - channel 1 frequency
    #theta - channel 1 phase
    
    #making sure dimensions are equal
    if (len(V_1)==len(V_2)==len(w)==len(theta)):
        n = len(V_1)
        print("Number of data points:",n)
        return True #returns true if all arrays are equal dimensions
    else:
        s = "You messed up imputting data"
        print(s)
        return False #lets me know if something here is wrong

## Module for calculating impedance

In [ ]:
def impedance(V_1,V_2,R_L):
    #V_1 - channel 1 voltage one
    #V_2 - channel 2 voltage two
    #R_L - load resistor
  
    #Calculating Impedance: Z=V1*R_L/V2
    Z = (V_1*R_L)/V_2
    print("Number of impedance values:",len(Z))
    
    return Z

## Module for Weighted Least Squares Fitting for find correlation between Impedance and Frequency

In [ ]:
def weighted_least_squares_fitting(x,y,sigma):
# Least squares fitting
    # y = a + b*x
    # n - number of trials
    #sigma - error in y
    
    #weighted parameter
    w = 1/sigma**2  #ARRAY

    #Creating arrays to hold various calculations for least squares fitting
    W = np.sum(w)
    WX = np.sum(w*x)
    WY = np.sum(w*y)
    WX_2 = np.sum(w*x*x)
    WXY = np.sum(w*x*y)
    delta = (W*WX_2) - (WX**2)
    
    #Calculate intercept (a)
    intercept = ((WX_2*WY)-(WX*WXY))/delta
    
    #Calculate slope (b)
    slope = ((W*WXY)-(WX*WY))/delta
    
    #Error in intercept
    int_err = math.sqrt(WX_2 / delta)
    
    #Error in slope
    slope_err = math.sqrt(W/delta)
    
    #Calculate X^2
    #X^2 = sum(W*(y-A-Bx)**2)
    box = y - intercept - (slope*x)
    print(box)
    box2 = box**2
    Chi_squared = np.sum(w*box2)

    
    return intercept,slope,int_err,slope_err, Chi_squared

## Error Propagation Module

In [ ]:
def impedance_error_propagation(load_resistor,volt1,volt2,sigma_1,sigma_2):
    #calc partials
    dzdv1 = load_resistor/volt2
    dzdv2 = -volt1*load_resistor/(volt2**2)
    
    v1_part = (dzdv1*sigma_1)**2
    v2_part = (dzdv2*sigma_2)**2
    
    #define sigma z array
    sigma_z = np.array([])
    
    for i in range(0,len(volt1)):
        sigma = math.sqrt(v1_part[i]+v2_part[i])
        sigma_z = np.append(sigma_z,sigma)
    
    return sigma_z

# Black Box A 

## Inputing Black Box A data

In [ ]:
# Inputting data into corresponding arrays

#First load resistor
#channel 1 voltage readings
V_a11 = np.array([11.5,11.3,10.5,9.52,9.68,9.68,9.36,
                 9.12,7.92,6.96,6.16,5.44,5.04,4.56,
                 4.24,3.84,2.68,1.54,0.656,0.424,
                 0.312,0.254,0.188,0.134])
sigma_Va11 = np.array([0.1,0.1,0.1,0.01,0.01,0.01,0.01,
                      0.01,0.01,0.01,0.01,0.01,0.01,0.01,
                      0.01,0.01,0.01,0.01,0.005,0.005,
                      0.005,0.005,0.005,0.005]) #volts

#channel 2 voltage readings
V_a21 = np.array([6.72,7.28,8.40,9.60,9.44,9.52,9.84,
                 10.0,10.9,11.5,11.9,12.2,12.4,12.5,
                 12.6,12.7,12.9,13.0,13.8,13.8,
                 13.4,13.4,13.8,13.8])
sigma_Va21 = np.array([0.01,0.01,0.01,0.01,0.01,0.01,0.01,
                      0.1,0.1,0.1,0.1,0.1,0.1,0.1,
                      0.1,0.1,0.1,0.1,0.1,0.1,
                      0.1,0.1,0.1,0.1]) #volts

#channel 1 frequency
w_a1 = np.array([91.16,101.4,129.5,167.2,160.8,162.2,173.5,
                184.4,233.4,288.4,341.5,397.5,431.0,491.6,
                538.8,601.7,892.1,1616.,3828.,6017.,
                8087.,10040.,13870.,19810.])


#channel 1 phase
phase_a1 = np.array([93.3,94.1,93.0,93.5,94.2,93.6,94.6,
                 93.5,94.4,93.6,94.6,94.0,93.6,95.7,
                 96.8,92.8,93.9,92.9,93.9,95.9,
                 96.6,100.,100.,104.])
sigma_phasea1 = 0.1 #degrees

#Second load resistor
#channel 1 voltage readings
V_a12 = np.array([2.34,1.52,1.10,0.896,0.792,0.668,0.572,
                 0.520,0.472,0.436,0.404,0.368,0.334,0.304,
                 0.282,0.266,0.254])
sigma_Va12 = np.array([0.01,0.01,0.01,0.005,0.005,0.005,0.005,
                      0.005,0.005,0.005,0.005,0.005,0.005,0.005,
                      0.005,0.005,0.005]) #volts

#channel 2 voltage readings
V_a22 = np.array([13.6,13.8,13.4,13.4,13.4,13.3,13.2,
                 13.3,13.2,13.2,13.2,13.3,13.2,13.2,
                 13.1,13.1,13.0])
sigma_Va22 = 0.1 #volts

#channel 1 frequency
w_a2 = np.array([10540.,16720.,22560.,27780.,31810.,37010.,43550.,
                48170.,53080.,58210.,62580.,67930.,74600.,82300.,
                89050.,94970.,98860.])

#channel 1 phase
phase_a2 = np.array([92.7,94.7,94.1,95.2,94.7,95.2,94.9,
                    97.3,95.0,95.0,97.6,96.7,98.0,98.3,
                    99.1,99.4,100.])
sigma_phasea2 = 0.1 #degrees

#making sure dimensions are equal
#first load resistor
A1 = array_check(V_a11,V_a21,w_a1,phase_a1)
if (A1 == True):
    print("All good!")
else:
    print("No bueno")
#second load resistor
A2 = array_check(V_a12,V_a22,w_a2,phase_a2)
if (A2 == True):
    print("All good!")
else:
    print("No bueno")
    
#Combining frequencies into a single array
w_a = np.append(w_a1,w_a2)
#Combining phase into a single array
phase_a = np.append(phase_a1,phase_a2)

## Calculating the Black Box A impedance

In [ ]:
#load resistors
R_L_1 = 10000.
R_L_2 = 1000.

#Calculating impedance from first resistor
Z_A1 = impedance(V_a11,V_a21,R_L_1)

#Calculating impedance from second resistor
Z_A2 = impedance(V_a12,V_a22,R_L_2)

#Combining impedance into single array
Z_A = np.append(Z_A1,Z_A2)
#print("Number of impedance values:",len(Z_A))
#print(Z_A)

#Finding z_err
#first load resistor
z_err_1 = impedance_error_propagation(R_L_1,V_a11,V_a21,sigma_Va11,sigma_Va21)
#second load resistor
z_err_2 = impedance_error_propagation(R_L_2,V_a12,V_a22,sigma_Va12,sigma_Va22)
#combine
zA_err = np.append(z_err_1,z_err_2)
#print(zA_err)

## Weighted Least Squares Fitting for Z^2 v. 1/w^2

In [ ]:
#Since the multimeter test resulted in open circuit, we can assume
#this is a capacitor in series with another element, 
#giving the following relationship
# Z^2 = R^2 + (1/C^2)(1/w^2)
x_a = 1/(w_a**2)
y_a = Z_A**2

#Error propagation of fit
#Assume no error from frequency, only from impedance
y_err = 2*Z_A*zA_err
print(y_err)

#Calculating intercept and slope
#Update to weighted and send sigma as well
#NEED TO FIND ERROR
A_a, B_a, Aa_err, Ba_err, Chi_squared_a = weighted_least_squares_fitting(x_a,y_a,y_err)

#Printing intercept, slope, X^2
print("R^2:",A_a, "+-",Aa_err)
print("1/C^2:",B_a,"+-",Ba_err)
print(Chi_squared_a)

## Theoretical relationships for capacitor in series with another linear element

In [ ]:
#From the least squares fitting, know that
#A_a = R^2
#B_a = 1/C^2

#Calculating resistor and capacitor values
R_a = math.sqrt(A_a)
#Error prop back, sigma R = sigma A /2*sqrt(A)
Ra_err = Aa_err/(2*math.sqrt(A_a))
C_a = 1/math.sqrt(B_a)
#Error prop, sigma c = sigma b/2 * B^-3/2
Ca_err = (Ba_err * B_a**(-3/2))/2

#print values
print("Box A resistor:",R_a,"+-",Ra_err,"Ohms")
print("Box A capacitance:",C_a,"+-", Ca_err,"Farads")

#theoretical equation for impedance
# Z = sqrt(R^2 + 1/(wC)^2) 
Z_a_theo = np.zeros(len(x_a),dtype=float)
#math.sqrt((R_a**2)+(1/((w_a*C_a)**2)))
for i in range(0,len(x_a)-1):  #start at 1 b/c we know f[0]
    Z_a_theo[i] = math.sqrt((R_a**2)+(1/((w_a[i]*C_a)**2)))
    
#theoretical equation for Z^2
#Z^2 = R^2 + (1/C^2)(1/w^2)
Z_a_2_theo = A_a + (B_a*x_a)

#actual tangent(theta)
tan_0_a = np.tan(phase_a)
#theoretical equation for phase
# tan(theta)= -1/RCw
tan_0_a_theo = -1/(R_a*C_a*w_a)

## Plotting Z^2 v. 1/w^2

In [ ]:
#adjust the size of the figure
fig = plt.figure(figsize=(10,10))

#Plotting least squares fitting
#Plotting Z^2_theo and Z^2 v. 1/w^2
plt.plot(x_a,Z_a_2_theo,label='Linear fit: Z^2 = R^2 + (1/C^2)(1/w^2), X^2 = 1640') 
plt.plot(x_a,y_a,'o',label='Z_a^2 v. 1/w_a^2') 

#Plot error
plt.errorbar(x=x_a,y=y_a,yerr = y_err, ls='none',marker='.',c='k',label='Error in Impedance^2')
#Plot labels
plt.xlabel(r'$1/Frequency^2(1/Hz^2)$')
plt.ylabel(r'$Impedance^2(Ohms^2)$')
plt.title('Linear Relationship of Capacitor and Resistor in series')
plt.legend(loc=1,framealpha=.85, frameon = True)
#Add tick marks and grid lines
plt.xticks()
plt.yticks()
plt.grid()
#Saving as png
plt.savefig('Z2versus1w2.png',bbox_inches="tight",dpi=600)

## Plotting Z v. w

In [ ]:
#adjust the size of the figure
fig = plt.figure(figsize=(10,10))

#Plotting Z_theo and Z v. w
plt.plot(w_a,Z_a_theo,label='Equation: Z=sqrt(R^2 + 1/(wC)^2)') 
plt.plot(w_a,Z_A,'o',label='Data: Z_a v. w_a') 

#plotting error
plt.errorbar(x=w_a,y=Z_A,yerr = zA_err, ls='none',marker='.',c='k',label='Error in Impedance')

#Plot labels
plt.xlabel(r'$Frequency(Hz)$')
plt.ylabel(r'$Impedance(Ohms)$')
plt.title('Z v. w for Capacitor and Resistor in series')
plt.legend(loc=1,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('Zversusw.png',bbox_inches="tight",dpi=600)

## Plotting tan(theta) v. 1/w

In [ ]:
#adjust the size of the figure
fig = plt.figure(figsize=(10,10))

#Plotting tan(theta)_theo and tan(theta) v. 1/w
plt.plot(1/w_a,tan_0_a_theo,label='Equation: tan(phase_a)=-1/(RCw)') 
plt.plot(1/w_a,tan_0_a,'o',label='Data: tan(phase_a) v. 1/w') 

#plot error, tan(0.1)
plt.errorbar(x=1/w_a,y=tan_0_a,yerr=0.1, ls='none',marker='.',c='k',label='Error in phase')
#Plot labels
plt.xlabel(r'$1/Frequency(1/Hz)$')
plt.ylabel(r'$Tangent(theta)$')
plt.title('Phase diagram for Capacitor and Resistor in series' )
plt.legend(loc=1,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('RCphase.png',bbox_inches="tight",dpi=600)

#ADD ERROR BARS ON PHASE DATA POINTS, NEED ANOTHER ARRAY
#RATIONALIZE BC MEASURES PHASE ON EXTREME FREQUENCY RANGED

In [ ]:
#plotting theta v. w instead
#adjust the size of the figure
fig = plt.figure(figsize=(10,10))

#Plotting tan(theta)_theo and tan(theta) v. 1/w
plt.plot(w_a,phase_a,'o',label='Data: phase_a v. w') 

#fit
theta = np.arctan(-1/(R_a*C_a*w_a))
plt.plot(w_a,theta,label='Equation: theta = arctan(-1/(R_a*C_a*w_a))') 

#plot error, arctan(0.1)=5.71
plt.errorbar(x=w_a,y=phase_a,yerr=5.71, ls='none',marker='.',c='k',label='Error in phase')
#Plot labels
plt.xlabel(r'$Frequency(Hz)$')
plt.ylabel(r'$Phase(degrees)$')
plt.title('Phase diagram for Capacitor and Resistor in series' )
plt.legend(loc=1,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('phasevw.png',bbox_inches="tight",dpi=600)

#ADD ERROR BARS ON PHASE DATA POINTS, NEED ANOTHER ARRAY
#RATIONALIZE BC MEASURES PHASE ON EXTREME FREQUENCY RANGED

In [ ]:
#plotting theta v. w instead
#adjust the size of the figure
fig = plt.figure(figsize=(10,10))

#Plotting tan(theta)_theo and tan(theta) v. 1/w
plt.plot(w_a,phase_a,'o',label='Data: phase_a v. w') 

#plot error
plt.errorbar(x=w_a,y=phase_a,yerr=0.1, ls='none',marker='.',c='k',label='Error in phase')
#Plot labels
plt.xlabel(r'$Frequency(Hz)$')
plt.ylabel(r'$Phase(degrees)$')
plt.title('Phase diagram for Capacitor and Resistor in series without fit' )
plt.legend(loc=1,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('phasevwdata.png',bbox_inches="tight",dpi=600)


# Black Box C

## Inputing Black Box C data
# Graph for inductor behavior weird, so moved last low point to res, may need to move back and write down how R, L values change

In [ ]:
# Inputting data into corresponding arrays

#Load resistance
R_L = 1000

#Low Frequency fit
#channel 1 voltage readings
V_c1_low = np.array([1.17,1.22,1.34,1.66,2.84,
                     4.44,6.04,8.00,8.96])
sigma_Vc1_low = 0.1

#channel 2 voltage readings
V_c2_low = np.array([10.5,10.6,10.5,10.5,10.6,
                    10.2,9.60,8.16,7.20])
sigma_Vc2_low = np.array([0.1,0.1,0.1,0.1,0.1,
                         0.1,0.01,0.01,0.01])

#channel 1 frequency
w_c_low = np.array([195.6,519.8,903.8,1391.,2931.,
                   4965.,6969.,9615.,10880.])

#channel 1 phase
phase_c_low = np.array([-170.,-156.,-148.,-139.,-122.,
                        -110.,-106.,-107.,-108.])
sigma_phaseclow = 1

#Resonant area
#not included in weighted fit
#channel 1 voltage readings
V_c1_res = np.array([10.8,11.0,11.1,11.2,11.2,11.2,11.1])
sigma_Vc1_res = 0.1

#channel 2 voltage readings
V_c2_res = np.array([3.36,2.64,2.22,1.64,1.14,1.60,1.96])
sigma_Vc2_res = 0.01

#channel 1 frequency
w_c_res = np.array([14880.,15530.,16020.,16640.,17510.,18220.,18730.])

#channel 1 phase
phase_c_res = np.array([-119.,-123.,-133.,-133.,-169.,-152.,153])
sigma_phasecres = 5

#High frequency fit
#channel 1 voltage readings
V_c1_high = np.array([11.0,10.5,8.96,8.32,7.04,6.08,5.60,5.12,
                     4.64,4.16,3.72,3.44,3.32,3.04,2.88,2.72])
sigma_Vc1_high = np.array([0.1,0.1,0.01,0.01,0.01,0.01,0.01,0.01,
                          0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01])

#channel 2 voltage readings
V_c2_high = np.array([2.86,4.68,7.08,8.08,9.20,9.52,10.6,11.0,
                     11.0,11.0,10.7,10.9,10.9,11.0,11.0,11.0])
sigma_Vc2_high = np.array([0.01,0.01,0.01,0.01,0.01,0.01,0.1,0.1,
                          0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])

#channel 1 frequency
w_c_high = np.array([19830.,22310.,27350.,30340.,
                     36740.,43030.,48170.,52740.,
                    59880.,66050.,71840.,77430.,
                    83160.,90580.,94250.,101800.])

#channel 1 phase
phase_c_high = np.array([112.,102.,100.,97.5,98.8,99.9,95.6,97.9,
                        97.0,97.6,98.5,100.,98.4,99.5,101.,101.])
sigma_phasechigh = 1
#making sure dimensions are equal
#first load resistor
C1 = array_check(V_c1_low,V_c2_low,w_c_low,phase_c_low)
if (C1 == True):
    print("All good!")
else:
    print("No bueno")
#second load resistor
C2 = array_check(V_c1_high,V_c2_high,w_c_high,phase_c_high)
if (C2 == True):
    print("All good!")
else:
    print("No bueno")
    
#Combining arrays for plotting
#Frequency
w_c_lowres = np.append(w_c_low,w_c_res)
w_c = np.append(w_c_lowres,w_c_high)
#Phase
phase_c_lowres = np.append(phase_c_low,phase_c_res)
phase_c = np.append(phase_c_lowres,phase_c_high)
    

## Calculating Black Box C Impedance

In [ ]:
#Calculating impedance for low frequency
Z_C_low = impedance(V_c1_low,V_c2_low,R_L) #cap acts like open ckt, inductor like wire?
#Calculating impedance for resonanct area (for plotting only)
Z_C_res = impedance(V_c1_res,V_c2_res,R_L)
#Calculating impedance for high frequency
Z_C_high = impedance(V_c1_high,V_c2_high,R_L) #higher inductor impedance, cap like wire

#Combining all to single array, for plotting
Z_C_lowres = np.append(Z_C_low,Z_C_res)
Z_C = np.append(Z_C_lowres,Z_C_high)
print(Z_C)

#Finding z_err
#low freq area
z_low_err = impedance_error_propagation(R_L,V_c1_low,V_c2_low,sigma_Vc1_low,sigma_Vc2_low)
#res area
z_res_err = impedance_error_propagation(R_L,V_c1_res,V_c2_res,sigma_Vc1_res,sigma_Vc2_res)
#high freq area
z_high_err = impedance_error_propagation(R_L,V_c1_high,V_c2_high,sigma_Vc1_high,sigma_Vc2_high)
#combine
err1 = np.append(z_low_err,z_res_err)
zC_err = np.append(err1,z_high_err)
print(zC_err)

## Weighted Least squares fitting for parallel resonant circuit

In [ ]:
#The overall relationship for a parallel resonant circuit
# Y^2 = 1/R^2 + (wC - 1/wL)^2
# tan(theta)=1/R(wL-1/wC)

#At low frequency, when w<<w_0 (resonant frequency),the capacitor acts like an open circuit, therefore
#the overall circuit exhibits inductor behavior, following
#Y^2 = 1/R^2 + (1/wL)^2
#tan(theta)=wL/R

#Determining inductor behavior
x_c_low = (1/w_c_low)**2
y_c_low = (1/Z_C_low)**2
n_c_low = len(x_c_low)

#Error propagation of fit
#Assume no error from frequency, only from impedance
y_err_low = 2*z_low_err / Z_C_low**3

#Calculating 1/R^2 and 1/L^2
A_c_low,B_c_low,AC_low_err,BC_low_err,Inductor_chi = weighted_least_squares_fitting(x_c_low,y_c_low,y_err_low)

#Calculating R_low and L
A_c_low = abs(A_c_low) #take absolute value to take square root
R_low = 1/math.sqrt(A_c_low)
#error propagation
R_low_err = (AC_low_err * A_c_low**(-3/2))/2
L = 1/math.sqrt(B_c_low)
#error propagation
L_err = (BC_low_err * B_c_low**(-3/2))/2

#Printing intercept and slope:
print("Low frequency, inductor behavior")
print("1/R^2:",A_c_low, "+-",AC_low_err)
print("Box C R_low:",R_low,"+-",R_low_err,"Ohms")
print("1/L^2:",B_c_low,"+-",BC_low_err)
print("Box C inductor:",L,"+-",L_err,"Henris")
print(Inductor_chi)

#At high frequency, when w>>w_0,the inductor's impedance dramatically increases 
#and the capacitor acts like a wire instead
#Capacitor behavior
#Y^2 = 1/R^2 + (wC)^2
#tan(phi)=-1/wCR

#Determining capacitor behavior
x_c_high = w_c_high**2
y_c_high = 1/(Z_C_high**2)
n_c_high = len(x_c_high)  #need to theoretical equations

#Error propagation of fit
#Assume no error from frequency, only from impedance
y_err_high = 2*z_high_err / Z_C_high**3

#Calculating 1/R^2 and C^2
A_c_high,B_c_high,AC_high_err,BC_high_err, Cap_chi = weighted_least_squares_fitting(x_c_high,y_c_high,y_err_high)

#Calculating R_high and C
A_c_high = abs(A_c_high) #take absolute value to take square root
R_high = 1/math.sqrt(A_c_high)
#Error propagation
R_high_err = (AC_high_err *A_c_high**(-3/2))/2
C = math.sqrt(B_c_high)
#error propagation
C_err = BC_high_err/(2*math.sqrt(B_c_high))

C_a = 1/math.sqrt(B_a)
#Error prop, sigma c = sigma b/2 * B^-3/2
Ca_err = (Ba_err * B_a**(-3/2))/2

#Printing intercept and slope:
print()
print("High frequency, capacitor behavior")
print("1/R^2:",A_c_high, "+-",AC_high_err,)
print("Box C R_high:",R_high,"+-",R_high_err,"Ohms")
print("C^2:",B_c_high,"+-",BC_high_err)
print("Box C capacitor:",C,"+-",C_err,"Farads")
print(Cap_chi)

## Theorectical Relationships for Resonant Circuits

In [ ]:
#theoretical equation for admittance with inductor behavorior
#Y^2 = 1/R^2 + (1/wL)^2
#tan(phi)=-R/wL

#Y^2 = 1/R^2 + (1/wL)^2
Y_ind_2_theo = (1/R_low**2)+((1/L**2)*(1/w_c_low**2))  #for inductor
#Y = sqrt(1/R^2 + (1/wL)^2)
Y_ind_theo = np.zeros(n_c_low,dtype=float)
for i in range(0,n_c_low-1):
    Y_ind_theo[i] = math.sqrt((1/R_low**2)+((1/L**2)*(1/w_c_low[i]**2)))
#tan(phi)=-R/wL
tan_phi_ind_theo = -R_low/w_c_low*L
#actual tan
tan_phi_ind = np.tan(phase_c_low)

#theoretical equation for admittance with capacitor behavior
#Y^2 = 1/R^2 + (wC)^2
#tan(phi)=wCR

#Y^2 = 1/R^2 + (wC)^2
Y_cap_2_theo = (1/R_high**2)+((C**2)*(w_c_high**2))  #for capacitor
#Y = sqrt(1/R^2 + (wC)^2)
Y_cap_theo = np.zeros(n_c_high,dtype=float)
for i in range(0,n_c_high-1):
    Y_cap_theo[i] = math.sqrt((1/R_high**2)+((C**2)*(w_c_high[i]**2)))
#tan(phi)=wCR
tan_phi_cap_theo = w_c_high*C*R_high
#actual tan
tan_phi_cap = np.tan(phase_c_high)

# Do these fits and then calc values for R,L,C
# Plug into and graph eq 120,121 (parallel res)

In [ ]:
#Relationship for low frequency - Inductor fit
#Y^2 = 1/R^2 + (1/wL)^2

#adjust the size of the figure
fig = plt.figure(figsize=(10,10))
plt.plot(1/w_c_low**2,Y_ind_2_theo, label='Fit: Y^2 = 1/R^2 + (1/wL)^2,X^2=1136')
plt.plot(1/w_c_low**2,1/Z_C_low**2,'o',label='Data: Y^2 v. 1/w^2') 

#plot error
plt.errorbar(x=1/w_c_low**2,y=1/Z_C_low**2,yerr=y_err_low, ls='none',marker='.',c='k',label='Error in Y^2')

#Plot labels
plt.xlabel(r'$1/Frequency^2(1/Hz^2)$')
plt.ylabel(r'$Admittance$')
plt.title('Inductor Behavior of Parallel Resonance Circuit')
plt.legend(loc=2,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('Inductorbehavior.png',bbox_inches="tight",dpi=600)

In [ ]:
#plotting |Y| = (1/R2 + (1/wL))½ 
#Y_ind_theo = math.sqrt((1/R_low**2)+((1/L**2)*(1/w_c_low[i]**2)))
#adjust the size of the figure
fig = plt.figure(figsize=(10,10))
plt.plot(w_c_low,Y_ind_theo, label='Equation: Y = sqrt(1/R^2 + (1/wL)^2)')
plt.plot(w_c_low,1/Z_C_low,'o',label='Data: Y v. w') 

#plot error
plt.errorbar(x=w_c_low,y=1/Z_C_low,yerr=y_err_low, ls='none',marker='.',c='k',label='Error in Y')

#Plot labels
plt.xlabel(r'$Frequency(Hz)$')
plt.ylabel(r'$Admittance$')
plt.title('Low Frequency Y v. w of Parallel Resonance Circuit')
plt.legend(loc=2,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('lowfrequency.png',bbox_inches="tight",dpi=600)

In [ ]:
#testing relationship for high frequency - capacitor behavior
#Y^2 = 1/R^2 + (wC)^2

#adjust the size of the figure
fig = plt.figure(figsize=(10,10))
plt.plot(w_c_high**2,Y_cap_2_theo,label='Fit: Y^2 = 1/R^2 + (wC)^2, X^2 = 445')
plt.plot(w_c_high**2,1/Z_C_high**2,'o',label='Data: Y^2 v. w^2')
         
#Plot error
plt.errorbar(x=w_c_high**2,y=1/Z_C_high**2,yerr=y_err_high,ls='none',marker='.',c='k',label='Error in Y^2' )

#Plot labels
plt.xlabel(r'$Frequency^2(Hz^2)$')
plt.ylabel(r'$Admittance^2$')
plt.title('Capacitor Behavior of Parallel Resonance Circuit')
plt.legend(loc=2,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('Capacitorfit.png',bbox_inches="tight",dpi=600)

In [ ]:
#plotting |Y| = (1/R2 + (wC)2)½
#Y_cap_theo[i] = math.sqrt((1/R_high**2)+((C**2)*(w_c_high[i]**2)))

#adjust the size of the figure
fig = plt.figure(figsize=(10,10))
plt.plot(w_c_high,Y_cap_theo,label='Equation: Y = sqrt(1/R^2 + (wC)^2)^1/2')
plt.plot(w_c_high,1/Z_C_high,'o',label='Data: Y v. w')
         
#Plot error
plt.errorbar(x=w_c_high,y=1/Z_C_high,yerr=y_err_high,ls='none',marker='.',c='k',label='Error in Y' )

#Plot labels
plt.xlabel(r'$Frequency(Hz)$')
plt.ylabel(r'$Admittance$')
plt.title('High Frequency Y v. w of Parallel Resonance Circuit')
plt.legend(loc=2,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('highfrequency.png',bbox_inches="tight",dpi=600)

In [ ]:
#plotting Z v. w for resonant circuit
#adjust the size of the figure
fig = plt.figure(figsize=(10,10))
plt.plot(w_c,Z_C,'o',label='Data: Z v. w') 
plt.plot(17500,9824.56140351,'o',label='Resonant frequency=17500Hz')
#finding resonant frequency via calculation
res_freq_calc = 1/math.sqrt(L*C)
plt.plot(res_freq_calc,9824.56140351,'o',label='Calculated Resonant frequency= 7695 Hz, 56%error from measured value')

#plot error
plt.errorbar(x=w_c,y=Z_C,yerr=zC_err,ls='none',marker='.',c='k',label='Error in Impedance')
#Plot labels
plt.xlabel(r'$Frequency(Hz)$')
plt.ylabel(r'$Impedance$')
plt.title('Relationship between Impedance and frequency in a parallel resonance circuit')
plt.legend(loc=1,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('Zvw_in_prc.png',bbox_inches="tight",dpi=600)

In [ ]:
#inductor phase
#tan(phi)=-R/wL
#tan_phi_ind_theo = -R_low/w_c_low*L
#actual tan
#tan_phi_ind = np.tan(phase_c_low)

#adjust the size of the figure
fig = plt.figure(figsize=(10,10))

#Plotting tan(phi)_theo and tan(phi) v. 1/w
plt.plot(1/w_c_low,tan_phi_ind_theo,label='Equation: tan(phase)=-R/wL') 
plt.plot(1/w_c_low,tan_phi_ind,'o',label='Data: tan(phi) v. 1/w') 

#plot error, tan(1)=0.02
plt.errorbar(x=1/w_c_low,y=tan_phi_ind,yerr=0.02, ls='none',marker='.',c='k',label='Error in tan(phase)')
#Plot labels
plt.xlabel(r'$1/Frequency(1/Hz)$')
plt.ylabel(r'$Tangent(phase)$')
plt.title('Phase diagram for Low Frequency Behavior' )
plt.legend(loc=1,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('LHPhase.png',bbox_inches="tight",dpi=600)


In [ ]:
#capacitor phase
#tan(phi)=wCR
#tan_phi_cap_theo = w_c_high*C*R_high
#actual tan
#tan_phi_cap = np.tan(phase_c_high)

#adjust the size of the figure
fig = plt.figure(figsize=(10,10))

#Plotting tan(phi)_theo and tan(phi) v. w
plt.plot(w_c_high,tan_phi_cap_theo,label='Equation: tan(phase)=RwC') 
plt.plot(w_c_high,tan_phi_cap,'o',label='Data: tan(phi) v. w') 

#plot error, tan(1)=0.02
plt.errorbar(x=w_c_high,y=tan_phi_cap,yerr=0.02, ls='none',marker='.',c='k',label='Error in tan(phase)')
#Plot labels
plt.xlabel(r'$Frequency(Hz)$')
plt.ylabel(r'$Tangent(phase)$')
plt.title('Phase diagram for High Frequency Behavior' )
plt.legend(loc=1,framealpha=.85, frameon = True)

#tickmarks and gridlines
plt.xticks()
plt.yticks()
plt.grid()

#Saving as png
plt.savefig('HFPhase.png',bbox_inches="tight",dpi=600)